--------------------------------------------------------------------------------------
![logo](https://4.bp.blogspot.com/-LAXjdvVCYCU/WxeQFKQ-1wI/AAAAAAAAACs/o8IJ1eLLAEwQYv2Az7EqQi9jODTqRx7wACK4BGAYYCw/s1000/tight%2Bbanner_with_description.png)

--------------------------------------------------------------------------------------
Please acknowledge <b>team cyda - Yeung Wong & Carrie Lo</b> when using the code

<b><font color='#3b5998'>If you find this script is helpful, please feel free to endorse us through Linkedin!</font></b>

<b>Linkedin:</b>

Carrie Lo - https://www.linkedin.com/in/carrielsc/

Yeung Wong - https://www.linkedin.com/in/yeungwong/

# 1. Preliminary

## 1.1 import libraries

In [1]:
#####################
# Kernal - Python 3 #
#####################

import pandas as pd
import folium
import json
from IPython.display import IFrame

#### Learn more about Folium: https://python-visualization.github.io/folium/modules.html

## 1.2 import data

In [2]:
data2 = pd.read_excel('Consolidated Data.xlsx',sheet_name='Voter Proportion By Age Group')
data2.head()

,區議會分區,18-35 (2019),36-65 (2019),>=66 (2019),18-35 (2016),36-65 (2016),>=66 (2016),18-35 (Diff),36-65 (Diff),>=66 (Diff),...,>=66 (% Change),18-35 (Proportion in 2019),36-65 (Proportion in 2019),>=66 (Proportion in 2019),18-35 (Proportion in 2016),36-65 (Proportion in 2016),>=66 (Proportion in 2016),18-35 (Change in Proportion),36-65 (Change in Proportion),>=66 (Change in Proportion)
0,中西區,22197,61678,26608,20682,61238,23084,1515,440,3524,...,0.152660,0.200909,0.558258,0.240833,0.196964,0.583197,0.219839,0.003945,-0.024939,0.020994
1,灣仔,12559,40401,21879,11816,40170,19660,743,231,2219,...,0.112869,0.167814,0.539839,0.292348,0.164922,0.560673,0.274405,0.002892,-0.020834,0.017943
2,東區,66532,174453,82740,62033,173464,71884,4499,989,10856,...,0.151021,0.205520,0.538893,0.255587,0.201811,0.564329,0.233860,0.003709,-0.025436,0.021728
3,南區,31665,83093,37236,29574,82004,32198,2091,1089,5038,...,0.156469,0.208331,0.546686,0.244983,0.205695,0.570359,0.223946,0.002636,-0.023673,0.021038
4,油尖旺,27970,70649,30531,25071,67588,27376,2899,3061,3155,...,0.115247,0.216570,0.547031,0.236400,0.208864,0.563069,0.228067,0.007706,-0.016039,0.008333


In [3]:
district_name = pd.read_excel('District Name.xlsx')

In [4]:
# Get a map json file
# HK : https://abhijeet.carto.com/tables/hkg_adm1/public
hk_geo = r'hkg_adm1.geojson'

# 2. Study the map json architecture & Prepare tooltip

## 2.1 Study the structure of the map json file

In [5]:
with open(hk_geo, encoding="utf8") as f: 
    map_data = f.readlines()
    map_data = [json.loads(line) for line in map_data]
map_data = map_data[0]

In [6]:
# Layer 1
[key for key in map_data]

['type', 'features']

In [7]:
# Layer 2 under Layer 1 - 'features'
[key for key in map_data['features'][0]]

['type', 'geometry', 'properties']

In [8]:
# Layer 3 under Layer 2 - 'geometry'
[key for key in map_data['features'][0]['geometry']]

['type', 'coordinates']

In [9]:
# Layer 3 under Layer 2 - 'properties'
[key for key in map_data['features'][0]['properties']]

['id_0',
 'iso',
 'name_0',
 'id_1',
 'name_1',
 'nl_name_1',
 'varname_1',
 'type_1',
 'engtype_1',
 'cartodb_id',
 'created_at',
 'updated_at']

In [10]:
print(map_data['features'][0]['properties'])

{'id_0': 102, 'iso': 'HKG', 'name_0': 'Hong Kong', 'id_1': 3, 'name_1': 'Islands', 'nl_name_1': None, 'varname_1': None, 'type_1': 'District', 'engtype_1': 'District', 'cartodb_id': 3, 'created_at': '2015-06-09T13:14:01Z', 'updated_at': '2015-06-09T13:14:01Z'}


In [11]:
json_map_file = []
for i in range(len(map_data['features'])):
    json_map_file.append(map_data['features'][i]['properties']['name_1'])
json_map_file = pd.DataFrame({'Sort_Index': range(len(map_data['features'])), 'Eng_name': json_map_file})
json_map_file

,Sort_Index,Eng_name
0,0,Islands
1,1,Tai Po
2,2,North
3,3,Sai Kung
4,4,Wong Tai Sin
5,5,Yau Tsim Mong
6,6,Central and Western
7,7,Sham Shui Po
8,8,Eastern
9,9,Southern


In [12]:
district_mapping = json_map_file.merge(district_name, on='Eng_name')
district_mapping

,Sort_Index,Eng_name,Chi_name
0,0,Islands,離島
1,1,Tai Po,大埔
2,2,North,北區
3,3,Sai Kung,西貢
4,4,Wong Tai Sin,黃大仙
5,5,Yau Tsim Mong,油尖旺
6,6,Central and Western,中西區
7,7,Sham Shui Po,深水埗
8,8,Eastern,東區
9,9,Southern,南區


In [13]:
data2 = data2.merge(district_mapping, left_on='區議會分區', right_on='Chi_name')
data2 = data2.sort_values(by=['Sort_Index']).reset_index(drop=True)

In [14]:
data2.head()

,區議會分區,18-35 (2019),36-65 (2019),>=66 (2019),18-35 (2016),36-65 (2016),>=66 (2016),18-35 (Diff),36-65 (Diff),>=66 (Diff),...,>=66 (Proportion in 2019),18-35 (Proportion in 2016),36-65 (Proportion in 2016),>=66 (Proportion in 2016),18-35 (Change in Proportion),36-65 (Change in Proportion),>=66 (Change in Proportion),Sort_Index,Eng_name,Chi_name
0,離島,22148,47251,14814,16408,40867,11364,5740,6384,3450,...,0.175911,0.239048,0.595390,0.165562,0.023952,-0.034301,0.010349,0,Islands,離島
1,大埔,50701,102559,34234,50492,98681,26547,209,3878,7687,...,0.182587,0.287344,0.561581,0.151076,-0.016930,-0.014582,0.031512,1,Tai Po,大埔
2,北區,52652,101010,32834,51433,97434,26205,1219,3576,6629,...,0.176057,0.293782,0.556537,0.149681,-0.011460,-0.014917,0.026376,2,North,北區
3,西貢,71232,157752,45309,63775,146486,35547,7457,11266,9762,...,0.165185,0.259450,0.595937,0.144613,0.000243,-0.020814,0.020572,3,Sai Kung,西貢
4,黃大仙,61338,136479,66223,55896,133031,60097,5442,3448,6126,...,0.250807,0.224460,0.534210,0.241330,0.007845,-0.017322,0.009477,4,Wong Tai Sin,黃大仙


## 2.2 Prepare tooltip

In [15]:
print(map_data['features'][0]['properties'])
print(map_data['features'][10]['properties'])

{'id_0': 102, 'iso': 'HKG', 'name_0': 'Hong Kong', 'id_1': 3, 'name_1': 'Islands', 'nl_name_1': None, 'varname_1': None, 'type_1': 'District', 'engtype_1': 'District', 'cartodb_id': 3, 'created_at': '2015-06-09T13:14:01Z', 'updated_at': '2015-06-09T13:14:01Z'}
{'id_0': 102, 'iso': 'HKG', 'name_0': 'Hong Kong', 'id_1': 4, 'name_1': 'Kowloon City', 'nl_name_1': None, 'varname_1': None, 'type_1': 'District', 'engtype_1': 'District', 'cartodb_id': 4, 'created_at': '2015-06-09T13:14:01Z', 'updated_at': '2015-06-09T13:14:01Z'}


In [16]:
# prepare the customised text
tooltip_text = []
for idx in range(len(data2)):
    tooltip_text.append(data2['Chi_name'][idx]+' '+ 
                        str(int(round(data2['18-35 (Proportion in 2019)'][idx]*100)))+'%')
tooltip_text

['離島 26%',
 '大埔 27%',
 '北區 28%',
 '西貢 26%',
 '黃大仙 23%',
 '油尖旺 22%',
 '中西區 20%',
 '深水埗 23%',
 '東區 21%',
 '南區 21%',
 '九龍城 21%',
 '葵青 24%',
 '觀塘 23%',
 '灣仔 17%',
 '沙田 24%',
 '荃灣 23%',
 '屯門 25%',
 '元朗 29%']

In [17]:
# prepare the customised text
tooltip_text2 = []
for idx in range(len(data2)):
    tooltip_text2.append(data2['Chi_name'][idx]+' '+ 
                        str(int(round(data2['>=66 (Proportion in 2019)'][idx]*100)))+'%')
tooltip_text2

['離島 18%',
 '大埔 18%',
 '北區 18%',
 '西貢 17%',
 '黃大仙 25%',
 '油尖旺 24%',
 '中西區 24%',
 '深水埗 25%',
 '東區 26%',
 '南區 24%',
 '九龍城 25%',
 '葵青 24%',
 '觀塘 26%',
 '灣仔 29%',
 '沙田 21%',
 '荃灣 20%',
 '屯門 19%',
 '元朗 15%']

In [18]:
# Append a tooltip column with customised text
for idx in range(len(tooltip_text)):
    map_data['features'][idx]['properties']['tooltip1'] = tooltip_text[idx]
    map_data['features'][idx]['properties']['tooltip2'] = tooltip_text2[idx]

In [19]:
# Sample checking 
print(map_data['features'][0]['properties'])

{'id_0': 102, 'iso': 'HKG', 'name_0': 'Hong Kong', 'id_1': 3, 'name_1': 'Islands', 'nl_name_1': None, 'varname_1': None, 'type_1': 'District', 'engtype_1': 'District', 'cartodb_id': 3, 'created_at': '2015-06-09T13:14:01Z', 'updated_at': '2015-06-09T13:14:01Z', 'tooltip1': '離島 26%', 'tooltip2': '離島 18%'}


In [20]:
# Save output
with open('hkg_mod.geojson', 'w') as output:
    json.dump(map_data, output)

# 3. Folium: A Map with Customised Tooltip

In [21]:
hk_geo = map_data
voter_proportion = data2[['Eng_name','18-35 (Proportion in 2019)']]

map2 = folium.Map([22.38, 114.15], tiles='cartodbpositron', zoom_start=10)

choropleth = folium.Choropleth(
    geo_data=hk_geo,
    name='選民比例',
    data=voter_proportion,
    columns=['Eng_name','18-35 (Proportion in 2019)'],
    key_on='feature.properties.name_1',
    fill_color='YlOrBr', # ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='2019年18-35歲選民 vs 18-35歲人口比例',
    highlight=True
).add_to(map2)

folium.LayerControl().add_to(map2)
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['tooltip1'], labels=False)
)

map2.save('map2.html')
IFrame('map2.html', width=700, height=450)

In [22]:
hk_geo = map_data
voter_proportion = data2[['Eng_name','>=66 (Proportion in 2019)']]

map3 = folium.Map([22.38, 114.15], tiles='cartodbpositron', zoom_start=10)

choropleth = folium.Choropleth(
    geo_data=hk_geo,
    name='選民比例',
    data=voter_proportion,
    columns=['Eng_name','>=66 (Proportion in 2019)'],
    key_on='feature.properties.name_1',
    fill_color='YlGnBu', # ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='2019年65歲以上選民 vs 65歲以上人口比例',
    highlight=True
).add_to(map3)

folium.LayerControl().add_to(map3)
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['tooltip2'], labels=False)
)

map3.save('map3.html')
IFrame('map3.html', width=700, height=450)